In [2]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor

# === 1. Cargar en chunks y filtrar hasta dic 2019 ===
usecols = ["product_id", "periodo", "tn"]
chunks = pd.read_csv("dataset_base_features.csv",
                     usecols=usecols,
                     parse_dates=["periodo"],
                     chunksize=500_000)

filtrados = []
total_rows = 0
chunk_count = 0

print("🟡 Procesando chunks...")
for chunk in chunks:
    chunk_count += 1
    original_rows = len(chunk)
    chunk = chunk[chunk["periodo"] <= "2019-12-01"]
    filtrados.append(chunk)
    total_rows += len(chunk)
    print(f"✅ Chunk {chunk_count}: {original_rows} filas → {len(chunk)} útiles (acumuladas: {total_rows})")

print("✅ Todos los chunks procesados. Total acumulado:", total_rows)

# === 2. Renombrar y preparar ===
df_train = pd.concat(filtrados, ignore_index=True)
df_train = df_train.rename(columns={"product_id": "item_id", "periodo": "timestamp", "tn": "target"})
df_train["item_id"] = df_train["item_id"].astype(str)

# Forzar timestamps al inicio de mes (por si hay algún desvío)
df_train["timestamp"] = df_train["timestamp"].dt.to_period("M").dt.to_timestamp()

# === 3. Crear enero 2020 sintético duplicando diciembre ===
print("📦 Generando enero sintético...")
enero_fake = df_train[df_train["timestamp"] == "2019-12-01"].copy()
enero_fake["timestamp"] = pd.to_datetime("2020-01-01")
enero_fake["target"] = pd.NA

# 🔁 Guardar enero_fake para inspección
enero_fake.to_csv("enero_fake.csv", index=False)
print("📝 Archivo 'enero_fake.csv' exportado para inspección.")

# === 4. Concatenar enero al dataset ===
df_train_con_enero = pd.concat([df_train, enero_fake], ignore_index=True)
df_train_con_enero = df_train_con_enero.sort_values(["item_id", "timestamp"])

# === 5. Entrenar modelo ===
print("⚙️ Entrenando modelo con AutoGluon...")
predictor = TimeSeriesPredictor(
    target="target",
    prediction_length=1,
    eval_metric="RMSE",
    freq="MS"  # ← CORRECCIÓN CLAVE
)
predictor.fit(train_data=df_train_con_enero, time_limit=600)
print("✅ Modelo entrenado.")

# === 6. Predecir febrero 2020 ===
print("🔮 Generando predicciones para febrero 2020...")
predicciones = predictor.predict(df_train_con_enero)

# === 7. Filtrar febrero 2020 ===
resultado = predicciones.reset_index()
print("🧪 Columnas disponibles en predicciones:", resultado.columns.tolist())

# Usamos la columna 'mean' como valor predicho
resultado = resultado[resultado["timestamp"] == pd.to_datetime("2020-02-01")]
resultado = resultado[["item_id", "mean"]].rename(columns={"item_id": "product_id", "mean": "tn_predicho"})

# === 8. Exportar resultado ===
resultado.to_csv("prediccion_feb2020_autogluon_from_enero.csv", index=False)
print("✅ Archivo exportado como 'prediccion_feb2020_autogluon_from_enero.csv'")



🟡 Procesando chunks...
✅ Chunk 1: 500000 filas → 500000 útiles (acumuladas: 500000)
✅ Chunk 2: 500000 filas → 500000 útiles (acumuladas: 1000000)
✅ Chunk 3: 500000 filas → 500000 útiles (acumuladas: 1500000)
✅ Chunk 4: 500000 filas → 500000 útiles (acumuladas: 2000000)
✅ Chunk 5: 500000 filas → 500000 útiles (acumuladas: 2500000)
✅ Chunk 6: 500000 filas → 500000 útiles (acumuladas: 3000000)
✅ Chunk 7: 500000 filas → 500000 útiles (acumuladas: 3500000)
✅ Chunk 8: 500000 filas → 500000 útiles (acumuladas: 4000000)
✅ Chunk 9: 500000 filas → 500000 útiles (acumuladas: 4500000)
✅ Chunk 10: 500000 filas → 500000 útiles (acumuladas: 5000000)
✅ Chunk 11: 500000 filas → 500000 útiles (acumuladas: 5500000)
✅ Chunk 12: 500000 filas → 500000 útiles (acumuladas: 6000000)
✅ Chunk 13: 500000 filas → 500000 útiles (acumuladas: 6500000)
✅ Chunk 14: 500000 filas → 500000 útiles (acumuladas: 7000000)
✅ Chunk 15: 500000 filas → 500000 útiles (acumuladas: 7500000)
✅ Chunk 16: 500000 filas → 500000 útiles (

C:\Users\waldo\AppData\Local\Temp\ipykernel_17916\3520336710.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_train_con_enero = pd.concat([df_train, enero_fake], ignore_index=True)
Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'c:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\AutogluonModels\ag-20250627_002622'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.23
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
GPU Count:          0
Memory Avail:       19.85 GB / 63.68 GB (31.2%)
Disk Space Avail:   1352.86 GB / 1862.26 GB (72.6%)

Fitting with arguments:
{'enable_ensemble': T

⚙️ Entrenando modelo con AutoGluon...


train_data with frequency 'IRREG' has been resampled to frequency 'MS'.
Provided train_data has 28860 rows (NaN fraction=2.7%), 780 time series. Median time series length is 37 (min=37, max=37). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-06-26 21:26:40
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'ChronosZeroShot[bolt_base]', 'ChronosFineTuned[bolt_small]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST', 'TiDE']
Training timeseries model SeasonalNaive. Training for up to 44.7s of the 581.7s of remaining time.
	nan           = Validation score (-RMSE)
	0.27    s     = Training runtime
	21.49   s     = Validation (prediction) runtime

✅ Modelo entrenado.
🔮 Generando predicciones para febrero 2020...


data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: SeasonalNaive


🧪 Columnas disponibles en predicciones: ['item_id', 'timestamp', 'mean', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']
✅ Archivo exportado como 'prediccion_feb2020_autogluon_from_enero.csv'
